In [3]:
from langgraph.graph import START, END, StateGraph
from langgraph.checkpoint.memory import InMemorySaver
from dotenv import load_dotenv
from langchain_groq import ChatGroq
from typing import TypedDict, Annotated
from langchain_core.messages import HumanMessage, BaseMessage
from langchain.messages import RemoveMessage
from langgraph.graph.message import add_messages
from CONFIG import GROQ_MODEL, TEMPERATURE

In [4]:
load_dotenv()
llm = ChatGroq(model=GROQ_MODEL, temperature=TEMPERATURE)

In [5]:
class state_class(TypedDict):
    messages: Annotated[list[BaseMessage], add_messages]

In [7]:
def chat_node(state: state_class):
    messages = state['messages']
    response = llm.invoke(messages)
    return {'messages': [response]}

def remove_messages(state: state_class):
    mess = state['messages']

    if len(mess) > 10:
        to_remove = mess[:6]
        return {'messages': [RemoveMessage(id=m.id) for m in to_remove]}
    return {}

In [16]:
checkpointer = InMemorySaver()

builder = StateGraph(state_class)
builder.add_node('chat_node', chat_node)
builder.add_node('remove_messages', remove_messages)

builder.add_edge(START, 'chat_node')
builder.add_edge('chat_node', 'remove_messages')
builder.add_edge('remove_messages', END)

graph = builder.compile(checkpointer=checkpointer)

In [17]:
config = {'configurable': {'thread_id': 'thread_09'}}

In [18]:
graph.invoke({'messages': HumanMessage(content="hi, how are you?")}, config)
graph.invoke({'messages': HumanMessage(content="do you know about me?")}, config)
graph.invoke({'messages': HumanMessage(content="ok my self adnan saeed")}, config)
graph.invoke({'messages': HumanMessage(content="what is your cut-date-knowledge")}, config)
graph.invoke({'messages': HumanMessage(content="which model do you used now?")}, config)
graph.invoke({'messages': HumanMessage(content="oh ok")}, config)
graph.invoke({'messages': HumanMessage(content="so at the end, what you want to ask from me?")}, config)

{'messages': [HumanMessage(content='what is your cut-date-knowledge', additional_kwargs={}, response_metadata={}, id='afd7f6b6-a8e5-47fc-b9e0-72bce2a0cab7'),
  AIMessage(content='My knowledge cutoff is currently December 2023. This means that my training data is based on information available up to December 2023, and I may not be aware of events, developments, or updates that have occurred after that date.\n\nIn other words, my knowledge is "frozen" as of December 2023, and I won\'t be able to provide information on very recent events, discoveries, or changes that have taken place after that date.\n\nHowever, my training data is based on a massive corpus of text, and I have been trained on a wide range of topics, including but not limited to:\n\n* General knowledge\n* History\n* Science\n* Technology\n* Arts\n* Culture\n* Entertainment\n* And many more!\n\nSo, while I may not have the very latest information, I can still provide a vast amount of knowledge and insights on a wide range o

In [20]:
config2 = {'configurable': {'thread_id': 'thread_010'}}

In [21]:
graph.invoke({'messages': HumanMessage(content="hi, how are you?")}, config2)
graph.invoke({'messages': HumanMessage(content="do you know about me?")}, config2)
graph.invoke({'messages': HumanMessage(content="ok my self adnan saeed")}, config2)

{'messages': [HumanMessage(content='hi, how are you?', additional_kwargs={}, response_metadata={}, id='48328c5c-6df1-4e59-b0d1-8a86b5a167a2'),
  AIMessage(content="Hello. I'm just a computer program, so I don't have feelings or emotions like humans do. However, I'm functioning properly and ready to assist you with any questions or tasks you may have. How can I help you today?", additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 49, 'prompt_tokens': 41, 'total_tokens': 90, 'completion_time': 0.083535905, 'completion_tokens_details': None, 'prompt_time': 0.003694166, 'prompt_tokens_details': None, 'queue_time': 0.008721771, 'total_time': 0.087230071}, 'model_name': 'llama-3.3-70b-versatile', 'system_fingerprint': 'fp_dae98b5ecb', 'service_tier': 'on_demand', 'finish_reason': 'stop', 'logprobs': None, 'model_provider': 'groq'}, id='lc_run--019bb36e-2ee2-7533-8115-5b9d6a8b6f24-0', usage_metadata={'input_tokens': 41, 'output_tokens': 49, 'total_tokens': 90}),
  Hu

In [34]:
print(f"config: It has {len(graph.get_state(config).values['messages'])} number of messages")
print(f"config2: It has {len(graph.get_state(config2).values['messages'])} number of messages")

config: It has 8 number of messages
config2: It has 6 number of messages
